__Copycat Metric Results__

In [10]:
import pandas as pd
import os
import numpy as np
import re
from tqdm import tqdm
import pickle
def norm(differences, ord):
    if ord==1:
        return np.sum(np.absolute(differences))
    if ord==2:
        return np.sqrt(np.sum(np.absolute(differences)**2))

In [11]:
df=pd.read_csv("/home/maximilian/Master/ol/metric_results.csv")

In [12]:
df

,baseline,metric,length,positions,experiment
0,bcso,our,1893,"[127, 131, 132, 133, 134, 136, 335, 703, 1006,...",bcso_nospeed_data_rep_3
1,bcso,kf,4846,"[4, 14, 15, 16, 17, 25, 42, 51, 52, 54, 76, 12...",bcso_nospeed_data_rep_3
2,bcoh,our,2098,"[129, 131, 135, 166, 1066, 1101, 1276, 1434, 1...",bcoh_nospeed_data_rep_3
3,bcoh,kf,4846,"[4, 14, 15, 16, 17, 25, 42, 51, 52, 54, 76, 12...",bcoh_nospeed_data_rep_3
4,arp,our,1855,"[135, 136, 137, 1024, 1727, 1728, 2228, 2438, ...",arp_nospeed_data_rep_3
5,arp,kf,4846,"[4, 14, 15, 16, 17, 25, 42, 51, 52, 54, 76, 12...",arp_nospeed_data_rep_3
6,keyframes,our,1786,"[137, 187, 197, 208, 233, 237, 250, 303, 588, ...",keyframes_threshold
7,keyframes,kf,4846,"[4, 14, 15, 16, 17, 25, 42, 51, 52, 54, 76, 12...",keyframes_threshold


In [13]:
for baseline in df["baseline"].unique():
    kf=list(map(int, df[(df["baseline"]==baseline) & (df["metric"]=="kf")]["positions"].to_numpy()[0].replace("[", "").replace("]", "").replace(" ","").split(",")))
    cc=list(map(int, df[(df["baseline"]==baseline) & (df["metric"]=="our")]["positions"].to_numpy()[0].replace("[", "").replace("]", "").replace(" ","").split(",")))
    count=0
    for value_kf in kf:
        if value_kf in cc:
            count+=1
    print("correspondences")
    print(baseline, count)
    print("percentage")
    print(baseline, f"{count/len(kf)*100:.2f}")

correspondences
bcso 67
percentage
bcso 1.38
correspondences
bcoh 51
percentage
bcoh 1.05
correspondences
arp 43
percentage
arp 0.89
correspondences
keyframes 37
percentage
keyframes 0.76


In [138]:
from PIL import Image
import os
import random
import cv2
from tqdm import tqdm
import numpy as np
def sort_key(s):
    s=s.strip(".jpg")
    return float(s)
def generate_video(path, save_path):
    images=sorted(os.listdir(path), key=sort_key)
    
    
    for i in tqdm(range(len(images))):
        image_object=np.array(Image.open(os.path.join(path,images[i])))
        image=Image.fromarray(image_object)
        image=np.array(image)
        if i==0:
            fps = 20
            final_shape=((1920, 1080))
            h,w,_=image.shape
            os.makedirs(os.path.dirname(save_path),exist_ok=True)
            video_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'XVID'),fps, (w, h))
        video_writer.write(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    video_writer.release()

In [139]:
main_root="/home/maximilian/cl_arp"
residuals={}
tuning_parameter_1=0.6
for root, dirs, files in tqdm(os.walk(main_root)):
    for file in files:
        if file.endswith(".pkl"):
            file_path=os.path.join(root, file)
            with open(file_path, "rb") as f:
                data=pickle.load(f)
            try:
                residual=norm(data["prev"]-data["curr"], ord=2)
            except TypeError:
                pass
            residuals.update({file_path: residual})
residual_values=np.array(list(residuals.values()))
mean=np.mean(residual_values)
std=np.std(residual_values)
for index, (file, residual_value) in tqdm(enumerate(residuals.items())):
    if residual_value<mean-std*tuning_parameter_1:
        rgb_folder=os.path.join(os.path.dirname(file), "with_rgb")
        generate_video(rgb_folder, save_path=os.path.join(main_root, "copycat_videos", f"{index}.avi"))
        with open(os.path.join(main_root, "closed_loop_copycat_cases.csv"), "a") as csvfile:
            csvfile.write(f"{file}\n")

42it [00:00, 4979.39it/s]
100%|██████████| 84/84 [00:03<00:00, 24.75it/s]
16it [00:56,  3.53s/it]


In [51]:
residuals

[0.39909005,
 0.08593563,
 0.8258511,
 0.13283864,
 0.32949677,
 0.50981313,
 0.19047014,
 0.6081346,
 0.6081346,
 0.74146825,
 0.27114025,
 0.27114025,
 0.76521343,
 0.76521343,
 0.76521343,
 0.76521343,
 0.4495973,
 0.79029614,
 0.16401032,
 0.036786754,
 0.40424374,
 0.43680313,
 2.6815019,
 1.1576914,
 0.52991265,
 0.098625,
 0.92457455,
 0.05141949,
 0.06503927,
 0.66967016,
 0.83599967,
 0.01629929,
 0.38340455,
 0.055159483,
 0.2796787,
 1.7596506,
 0.33925766,
 0.43884975,
 0.03663834,
 0.18676788,
 0.6299309,
 0.117241696,
 0.27298796,
 0.26100683,
 0.6520736,
 0.09165134,
 0.27672446,
 1.2901827,
 0.38602126,
 0.022754489,
 0.7390922,
 0.50544924,
 0.27691507,
 0.27691507,
 0.5380503,
 0.011213032,
 0.36691755,
 0.19279836,
 0.4628256,
 0.94119954,
 0.015410067,
 1.1392314,
 0.1874046,
 0.5113955,
 1.0713725,
 0.044874374,
 0.4283917,
 0.07946182,
 0.40052795,
 0.3662524,
 0.080483355,
 0.04003524,
 1.1751913,
 3.6380992,
 0.041747376,
 1.3089017,
 2.5418682,
 0.091982245,
 0

__Analysis Script__

In [31]:
path="/home/maximilian/Master/carla_garage/logs"

In [32]:
summary="id_000.csv"

In [33]:
df=pd.read_csv(os.path.join(path, summary))

In [34]:
df

,town,baseline,experiment,setting,traffic,weather,start,target,route_completion,outside_route,stops_ran,inroute,lights_ran,collision,duration,timeout_blocked


In [35]:
df=df[df["baseline"]=="bcso"]

In [36]:
df

,town,baseline,experiment,setting,traffic,weather,start,target,route_completion,outside_route,stops_ran,inroute,lights_ran,collision,duration,timeout_blocked


In [37]:
threshold=.5

In [38]:
# (df["timeout"]-df["final_time"]<threshold).sum()
timeouts=(df["timeout"]==1).sum()

KeyError: 'timeout'

In [ ]:
timeouts

0

In [ ]:
#collisions=((df["end_other_collision"]) | (df["end_pedestrian_collision"]) | (df["end_vehicle_collision"])).sum()
collisions=(df["collision"]==1).sum()

In [ ]:
not_finished=((df["route_completion"]<90)&(df["collision"]==0) &(df["timeout"]==0)).sum()

In [ ]:
outside_route=((df["route_completion"]>90)&(df["collision"]==0) &(df["timeout"]==0)&(df["outside_route"]>0)).sum()

In [ ]:
outside_route

2

In [ ]:
not_finished

0

In [ ]:
collisions/len(df)

0.08

In [ ]:
success=len(df)-timeouts-collisions-not_finished-outside_route

In [ ]:
success/len(df)

0.84

In [53]:
from tqdm import tqdm
import regex as re
import os
import pandas as pd

ModuleNotFoundError: No module named 'coil_utils'

In [44]:
print("Started scanning...")
result_files = {}
for root, dirs, files in os.walk("/home/maximilian/Master/carla_garage/test"):
    for dir in dirs:
        eval_reps = os.path.join(root, dir)
        if re.findall(".*_.*", dir):
            for rep_root, rep_dirs, rep_files in os.walk(eval_reps):
                for rep_dir in rep_dirs:
                    file_lst=[]
                    rep_path = os.path.join(eval_reps, rep_dir)
                    if rep_dir == "results":
                        for filename in tqdm(os.listdir(rep_path)):
                            file_lst.append(os.path.join(rep_path, filename))
                        result_files[dir] = file_lst

Started scanning...


100%|██████████| 2/2 [00:00<00:00, 1420.59it/s]


In [45]:
weathers = {"train": [1, 6, 10, 14], "test": [3,8]}
def weather_mapping(value):
    for key, value_lst in weathers.items():
        if value in value_lst:
            return key

In [46]:
result_files

{'nocrash_e1_30': ['/home/maximilian/Master/carla_garage/test/nocrash_e1_30/results/bcso_nocrash_e-baseline-bcso_speed-0_prevnum-0_backbone-unrolling_head-1_lossweights-0.33,0.33,0.33_tr-0_w-train_t-Town01_r-1_t-repetition_0_s-02_withheld_o-nocrash_Town01_split_10.csv',
  '/home/maximilian/Master/carla_garage/test/nocrash_e1_30/results/bcso_nocrash_e-baseline-bcso_speed-0_prevnum-0_backbone-unrolling_head-1_lossweights-0.33,0.33,0.33_tr-0_w-train_t-Town01_r-1_t-repetition_0_s-02_withheld_o-nocrash_Town01_split_1.csv']}

In [56]:
def get_ablations_dict():
    return {"head":0, "speed":0, "prevnum":0, "backbone":0, "lossweights": [0, 0, 0]}

In [57]:
df_lst=[]
for eval_rep, path_lst in tqdm(result_files.items()):
    for path in path_lst:
        eval_results = pd.read_csv(path)
        df_lst.append(eval_results)
df = pd.concat(df_lst, ignore_index=True)
df["success"] = (df["timeout_blocked"] == 0) & (df["collision"] == 0)
df["success"] = df["success"].astype("int")
df["weather"] = df["weather"].map(lambda x: weather_mapping(x))
ablations_dict=get_ablations_dict()

groups = df.groupby(["baseline", "town", "traffic", "weather", *ablations_dict]).agg(
    timeout_mean=("timeout_blocked", "mean"),
    timeout_std=("timeout_blocked", "std"),
    collisions_mean=("collision", "mean"),
    collisions_std=("collision", "std"),
    success_mean=("success", "mean"),
    success_std=("success", "std"),
)


100%|██████████| 1/1 [00:00<00:00, 147.60it/s]


In [58]:
groups=(groups*100).round(2)

In [59]:
groups

,,,,,,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,town,traffic,weather,head,speed,prevnum,backbone,lossweights,,,,,,
bcso,Town01,0,train,1,0,0,unrolling,"[0.33, 0.33, 0.33]",0.0,0.0,0.0,0.0,100.0,0.0


In [11]:
pd.set_option('display.max_rows', 500)

__Weather: Train Weather,\
Traffic: Empty,\
Town: Training Town__

In [12]:
groups.loc[(slice(None), slice(None), "Town01", 0, "train", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town01,0,0.33,0.00,7.11,0.19,92.56,0.19
bcoh,bcoh_nospeed_data_rep_3,Town01,0,13.78,0.69,19.22,1.35,67.00,1.33
bcso,bcso_nospeed_data_rep_3,Town01,0,2.11,0.19,5.78,0.84,92.11,0.69
keyframes,keyframes_threshold,Town01,0,7.67,2.52,10.33,2.08,82.00,3.46


__Weather: Train Weather,\
Traffic: Regular,\
Town: Training Town__

In [13]:
groups.loc[(slice(None), slice(None), "Town01", 1, "train", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town01,1,0.22,0.19,6.33,0.88,93.44,0.69
bcoh,bcoh_nospeed_data_rep_3,Town01,1,9.44,0.51,22.33,1.00,68.22,0.77
bcso,bcso_nospeed_data_rep_3,Town01,1,0.33,0.00,8.22,1.35,91.44,1.35
keyframes,keyframes_threshold,Town01,1,3.67,2.52,13.67,1.15,82.67,3.51


__Weather: Train Weather,\
Traffic: Dense,\
Town: Training Town__

In [14]:
groups.loc[(slice(None), slice(None), "Town01", 2, "train", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town01,2,1.78,0.19,13.78,1.39,84.44,1.58
bcoh,bcoh_nospeed_data_rep_3,Town01,2,9.33,0.33,29.56,2.41,61.11,2.41
bcso,bcso_nospeed_data_rep_3,Town01,2,1.56,0.38,19.33,0.33,79.11,0.51
keyframes,keyframes_threshold,Town01,2,7.33,2.52,20.67,0.58,72.00,2.65


__Weather: Test Weather,\
Traffic: Empty,\
Town: Training Town__

In [15]:
groups.loc[(slice(None), slice(None), "Town01", 0, "test", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town01,0,0.00,0.00,8.44,1.02,91.56,1.02
bcoh,bcoh_nospeed_data_rep_3,Town01,0,7.33,0.67,23.33,0.67,69.33,0.00
bcso,bcso_nospeed_data_rep_3,Town01,0,0.22,0.38,4.44,1.39,95.33,1.33
keyframes,keyframes_threshold,Town01,0,4.67,1.15,10.67,3.06,84.67,3.06


__Weather: Test Weather,\
Traffic: Regular,\
Town: Training Town__

In [16]:
groups.loc[(slice(None), slice(None), "Town01", 1, "test", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town01,1,0.00,0.00,8.44,1.02,91.56,1.02
bcoh,bcoh_nospeed_data_rep_3,Town01,1,4.67,1.76,26.89,3.91,68.44,2.78
bcso,bcso_nospeed_data_rep_3,Town01,1,0.00,0.00,6.22,0.38,93.78,0.38
keyframes,keyframes_threshold,Town01,1,2.67,1.15,10.67,1.15,86.67,1.15


__Weather: Test Weather,\
Traffic: Dense,\
Town: Training Town__

In [17]:
groups.loc[(slice(None), slice(None), "Town01", 2, "test", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town01,2,2.00,1.15,12.67,2.31,85.33,2.00
bcoh,bcoh_nospeed_data_rep_3,Town01,2,9.11,1.02,27.33,2.40,63.56,3.08
bcso,bcso_nospeed_data_rep_3,Town01,2,2.00,0.67,18.00,1.76,80.00,1.15
keyframes,keyframes_threshold,Town01,2,6.67,1.15,17.33,3.06,76.00,3.46


__Weather: Test Weather,\
Traffic: Empty,\
Town: Test Town__

In [18]:
groups.loc[(slice(None), slice(None), "Town02", 0, "test", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town02,0,2.00,0.67,16.00,0.67,82.00,0.67
bcoh,bcoh_nospeed_data_rep_3,Town02,0,20.44,2.69,22.89,1.39,56.67,1.33
bcso,bcso_nospeed_data_rep_3,Town02,0,0.89,0.77,4.44,1.02,94.67,1.15
keyframes,keyframes_threshold,Town02,0,27.33,1.15,16.67,4.62,56.00,5.29


__Weather: Test Weather,\
Traffic: Regular,\
Town: Test Town__

In [19]:
groups.loc[(slice(None), slice(None), "Town02", 1, "test", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town02,1,4.00,0.00,16.44,1.02,79.56,1.02
bcoh,bcoh_nospeed_data_rep_3,Town02,1,19.33,1.15,28.00,1.15,52.67,0.00
bcso,bcso_nospeed_data_rep_3,Town02,1,1.11,1.02,10.89,0.77,88.00,0.67
keyframes,keyframes_threshold,Town02,1,22.00,5.29,25.33,2.31,52.67,7.57


__Weather: Test Weather,\
Traffic: Dense,\
Town: Test Town__

In [20]:
groups.loc[(slice(None), slice(None), "Town02", 2, "test", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town02,2,30.22,4.02,27.33,3.53,42.44,1.68
bcoh,bcoh_nospeed_data_rep_3,Town02,2,46.22,3.36,36.00,3.46,17.78,2.14
bcso,bcso_nospeed_data_rep_3,Town02,2,21.56,2.04,37.56,2.52,40.89,1.39
keyframes,keyframes_threshold,Town02,2,54.67,8.33,30.00,5.29,15.33,3.06


__Weather: Train Weather,\
Traffic: Empy,\
Town: Test Town__

In [21]:
groups.loc[(slice(None), slice(None), "Town02", 0, "train", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town02,0,2.33,0.33,20.11,0.51,77.56,0.51
bcoh,bcoh_nospeed_data_rep_3,Town02,0,28.78,1.17,22.00,1.76,49.22,1.26
bcso,bcso_nospeed_data_rep_3,Town02,0,0.33,0.58,9.78,1.26,89.89,1.58
keyframes,keyframes_threshold,Town02,0,27.33,1.53,26.33,2.52,46.33,4.04


__Weather: Train Weather,\
Traffic: Regular,\
Town: Test Town__

In [22]:
groups.loc[(slice(None), slice(None), "Town02", 1, "train", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town02,1,3.78,0.19,21.44,0.51,74.78,0.69
bcoh,bcoh_nospeed_data_rep_3,Town02,1,26.33,0.88,28.22,2.50,45.44,3.37
bcso,bcso_nospeed_data_rep_3,Town02,1,1.67,0.88,12.00,0.67,86.33,0.58
keyframes,keyframes_threshold,Town02,1,24.00,3.00,30.67,5.69,45.33,2.89


__Weather: Train Weather,\
Traffic: Dense,\
Town: Test Town__

In [23]:
groups.loc[(slice(None), slice(None), "Town02", 2, "train", "02_withheld")].reset_index(level=[4,5], drop=True)

,,,,timeout_mean,timeout_std,collisions_mean,collisions_std,success_mean,success_std
baseline,experiment,town,traffic,,,,,,
arp,arp_nospeed_data_rep_3,Town02,2,31.22,3.29,28.00,1.86,40.78,2.01
bcoh,bcoh_nospeed_data_rep_3,Town02,2,43.56,0.51,38.67,0.88,17.89,0.84
bcso,bcso_nospeed_data_rep_3,Town02,2,25.33,2.60,36.56,1.39,38.11,1.84
keyframes,keyframes_threshold,Town02,2,45.33,2.52,42.33,2.52,12.33,2.52


In [24]:
#groups.to_csv(os.path.join("/home/maximilian/Master/first_large_evaluation_all_settings_all_baselines_partially_wrong_speed_input", os.path.join("/home/maximilian/Master/first_large_evaluation_all_settings_all_baselines_partially_wrong_speed_input","combined.csv")))